## GAP Data Analytics, Package Distribution

This Jupyter Notebook allows for automatically pulling information based on the GAP package distribution repository from GitHub, through metadata from all GAP packages. 

In [1]:
# Import required libraries and packages
import os
import sys
import requests

# Get current working directory and append parent directory for module imports
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
sys.path.append(parent_dir)

# Import modules from other project scripts
from data_constants import *

### Updates to Package Distribution

To check for more detailed updates to packages distributed under GAP, there is a repository under the gap-system organisation on GitHub that will scan for updates in the GitHub hosted packages that fall under GAP. As such, this information can be utilised to analyse what packages should be considered in a new release of the system. By comparing current versions to what is in the repository, analysis can be made to determine what the next version will look like.

In [ ]:
# Define repositories that are public for gap-packages organisation on GitHub
org = g.get_organization(ORG_NAME_SYSTEM)
repo = org.get_repo(DISTRO_REPO)

In [ ]:
# Function to get the latest relesase of GAP and the commit for this version
def get_latest_release():
    repo_url = "https://api.github.com/repos/gap-system/PackageDistro/releases/latest"
    response = requests.get(repo_url)
    latest_release = response.json()
    latest_version = latest_release.get("name")
    version_commit = latest_release.get("target_commitish")
    return latest_version, version_commit

latest_gap_release, version_commit = get_latest_release()
print(f"The latest version of GAP is: {latest_gap_release} and it has the commit {version_commit}")

In [ ]:
# Function to get the version from a meta.json file
def get_version_from_meta(meta_json_url):
    response = requests.get(meta_json_url)
    meta_json = response.json()
    version = meta_json.get("Version")
    return version

In [ ]:
# Function to get all meta.json files and versions based on the branch
def get_meta(branch):
    api_url = f"https://api.github.com/repos/gap-system/PackageDistro/contents/packages?ref={branch}"
    response = requests.get(api_url)
    package_folders = response.json()

    meta_json_data = []
    for folder in package_folders:
        if folder.get("type") == "dir":
            package_name = folder.get("name")
            meta_json_url = f"https://raw.githubusercontent.com/gap-system/PackageDistro/{branch}/packages/{package_name}/meta.json"
            version = get_version_from_meta(meta_json_url)
            meta_json_data.append((package_name, version))

    return meta_json_data

In [ ]:
# Function to retrieve pull requests with specified labels
def get_open_pull_requests(labels):
    api_url = "https://api.github.com/repos/gap-system/PackageDistro/pulls"

    params = {
        "state": "open",
        "labels": ",".join(labels)
    }

    response = requests.get(api_url, params=params)
    pull_requests = response.json()
    return pull_requests

In [ ]:
# Get meta.json files and versions for the latest release
latest_meta = get_meta(get_latest_release()[1])
print("Package versions in the latest GAP release:")
for name, version in latest_meta:
    print("Name:", name)
    print("Version:", version)

In [ ]:
# Get meta.json files and versions for the main branch
main_meta = get_meta("main")
print("Current package version in the main branch:")
for name, version in main_meta:
    print("Name:", name)
    print("Version:", version)

In [ ]:
# Compare versions and print package names if they are different
# For the packages with different version, the package in the main branch will be the new version in the next release
print("Packages with different versions:")
for latest_package, latest_version in latest_meta:
    for main_package, main_version in main_meta:
        if latest_package == main_package and latest_version != main_version:
            print(f"Package: {latest_package}")
            print(f"Latest Version: {latest_version}")
            print(f"Main Branch Version: {main_version}")
            print("---")

In [ ]:
# Retrieve pull requests with specified labels
labels = ["automatic pr", "new package", "update package"]
pull_requests = get_open_pull_requests(labels)

# Extract the package names from pull requests
package_names = {pr["head"]["ref"].split("/")[1] for pr in pull_requests}
print(package_names)

In [ ]:
# Packages that were in the latest release that might also be in the next
print("Packages in unmerged pull requests:")
for package_name in package_names:
    for latest_package, latest_version in latest_meta:
        if package_name == latest_package:
            print(f"Package: {package_name}")
            print(f"Latest Release Version: {latest_version}")
            print("---")

In [ ]:
# All packages that might be in the next release based on open PRs
print("Packages in unmerged pull requests:")
for package_name in package_names:
    print(f"Package: {package_name}")